In [9]:
using Revise

In [10]:
using PyCall, PyPlot, JuMP
using ClimateMARGO

In [11]:
using ClimateMARGO.Diagnostics
using ClimateMARGO.Models

In [12]:
using Combinatorics

# Optimizating the Global Climate Action Policy Portfolio

In [15]:
model_og = deepcopy(ClimateModel(ClimateMARGO.IO.included_configurations["default"]));
max_deployment_og = Dict("mitigate"=>1., "remove"=>1., "geoeng"=>1., "adapt"=>0.4)

Dict{String,Float64} with 4 entries:
  "geoeng"   => 1.0
  "mitigate" => 1.0
  "remove"   => 1.0
  "adapt"    => 0.4

In [16]:
controls = collect(keys(max_deployment_og));

In [17]:
control_permutations = combinations(controls) |> collect

15-element Array{Array{String,1},1}:
 ["geoeng"]
 ["mitigate"]
 ["remove"]
 ["adapt"]
 ["geoeng", "mitigate"]
 ["geoeng", "remove"]
 ["geoeng", "adapt"]
 ["mitigate", "remove"]
 ["mitigate", "adapt"]
 ["remove", "adapt"]
 ["geoeng", "mitigate", "remove"]
 ["geoeng", "mitigate", "adapt"]
 ["geoeng", "remove", "adapt"]
 ["mitigate", "remove", "adapt"]
 ["geoeng", "mitigate", "remove", "adapt"]

In [197]:
function exp_short_name(control_permutation)
    tmp_name = ""
    if "mitigate" in control_permutation; tmp_name = string(tmp_name, "M"); end;
    if "remove" in control_permutation; tmp_name = string(tmp_name, "R"); end;
    if "geoeng" in control_permutation; tmp_name = string(tmp_name, "G"); end;
    if "adapt" in control_permutation; tmp_name = string(tmp_name, "A"); end;
    return tmp_name
end;

In [191]:
parameter_permutations = Dict(
    :economics => [:ρ, :mitigate_cost, :remove_cost, :geoeng_cost, :adapt_cost],
    :physics => [:B, :r, :κ]
)

Dict{Symbol,Array{Symbol,1}} with 2 entries:
  :physics   => [:B, :r, :κ]
  :economics => [:ρ, :mitigate_cost, :remove_cost, :geoeng_cost, :adapt_cost]

In [203]:
results = Dict()

for (submodel, parameter_list) in parameter_permutations
    for parameter in parameter_list
        for (perturb_name, perturb_value) in zip(["low", "high"], [0.5, 1.5])
            
            exp = string(submodel, "-", parameter, "-", perturb_name)
            results[exp] = Dict()
            for control_permutation in control_permutations
                model = deepcopy(ClimateModel(ClimateMARGO.IO.included_configurations["default"]));
                
                setfield!(getfield(model, submodel), parameter, getfield(getfield(model_og, submodel), parameter)*perturb_value)
                # Assert that some controls not be deployed
                max_deployment = copy(max_deployment_og)
                for control in controls
                    if ~(control in control_permutation)
                        max_deployment[control] = 0.
                        # Override initial condition constraints
                        if ~isnothing(getfield(model.economics, Symbol(string(control,"_init"))))
                            setfield!(model.economics, Symbol(string(control,"_init")), 0.)
                        end
                    end
                end
                m = ClimateMARGO.Optimization.optimize_controls!(model, max_deployment = max_deployment, temp_goal=2.0, print_raw_status=false);
                
                results[exp][exp_short_name(control_permutation)] = Dict(
                    "model" => model,
                    "NPV" => net_present_cost(model, M=true, R=true, G=true, A=true),
                    "status" => raw_status(m)
                )
            end
        end
    end
end

In [266]:
exp_name = "physics-B-low"

_keys = deepcopy([val for val in collect(keys(results[exp_name]))]);
_status = deepcopy([val["status"] for val in collect(values(results[exp_name]))])
_vals = deepcopy([val["NPV"] for val in collect(values(results[exp_name]))]);

_vals[_status .== "Infeasible_Problem_Detected"] .= NaN
idx = sortperm(_vals);
#_keys = _keys[idx]; _vals = _vals[idx]; _status = _status[idx];

In [279]:
_keys

15-element Array{String,1}:
 "MG"
 "MGA"
 "RA"
 "M"
 "RG"
 "A"
 "G"
 "GA"
 "MRG"
 "MR"
 "RGA"
 "MA"
 "R"
 "MRGA"
 "MRA"

In [274]:
findall(x->x=="M", _keys)

1-element Array{Int64,1}:
 4

In [309]:
ranking = Dict()
for perm in control_permutations; ranking[exp_short_name(perm)] = 0.; end;
    
highest_rank = Dict()
for perm in control_permutations; highest_rank[exp_short_name(perm)] = 1; end;

lowest_rank = Dict()
for perm in control_permutations; lowest_rank[exp_short_name(perm)] = 15; end;

for (exp_k, exp_v) in results
    
    _keys = deepcopy([val for val in collect(keys(exp_v))]);
    _status = deepcopy([val["status"] for val in collect(values(exp_v))])
    _vals = deepcopy([val["NPV"] for val in collect(values(exp_v))]);

    _vals[_status .== "Infeasible_Problem_Detected"] .= NaN
    idx = sortperm(_vals);
    _keys = _keys[idx]; _vals = _vals[idx]; _status = _status[idx];
    
    for (perm_k, perm_v) in exp_v
        rank = findall(x->x==perm_k, _keys)[1]
        ranking[perm_k] += rank/ length(results)
        if rank > highest_rank[perm_k];
            highest_rank[perm_k] = rank;
        end
        if rank < lowest_rank[perm_k];
            lowest_rank[perm_k] = rank
        end
    end
end

In [401]:
_highest_rank = collect(values(highest_rank))
_lowest_rank = collect(values(lowest_rank))
_names = collect(keys(lowest_rank))
idx = sortperm(collect(values(lowest_rank)))
_highest_rank = _highest_rank[idx]; _lowest_rank = _lowest_rank[idx]; _names = _names[idx]; 

ranked_clusters = []
for (k, h, l) in zip(_names, _highest_rank, _lowest_rank)
    #println(k, " ", _names[l .>= _lowest_rank], " ", _names[h .<= _highest_rank])
    always_lower_than = _names[h .<= _lowest_rank]
    always_higher_than = _names[l .>= _highest_rank]
    println(k, " ", [name for name in _names if ((name in always_lower_than) & (name in always_higher_than))])
end

MRGA ["MRGA"]
MGA Any[]
MRG Any[]
MG ["MG"]
RGA Any[]
MRA Any[]
GA Any[]
MR Any[]
RG Any[]
MA Any[]
M Any[]
G Any[]
RA Any[]
A ["A"]
R ["R"]


In [383]:
_names[1 .<= _lowest_rank]

Dict{Any,Any} with 15 entries:
  "MG"   => 4
  "MGA"  => 3
  "RA"   => 13
  "M"    => 13
  "RG"   => 11
  "A"    => 14
  "G"    => 12
  "GA"   => 10
  "MRG"  => 3
  "MR"   => 10
  "RGA"  => 9
  "MA"   => 11
  "R"    => 15
  "MRGA" => 1
  "MRA"  => 9

In [385]:
lowest_rank

Dict{Any,Any} with 15 entries:
  "MG"   => 4
  "MGA"  => 2
  "RA"   => 12
  "M"    => 8
  "RG"   => 7
  "A"    => 14
  "G"    => 8
  "GA"   => 6
  "MRG"  => 2
  "MR"   => 6
  "RGA"  => 5
  "MA"   => 7
  "R"    => 15
  "MRGA" => 1
  "MRA"  => 5

In [314]:
always_higher = Dict()
always_lower = Dict()

Dict{Any,Any}()

In [311]:
lower_rank

LoadError: UndefVarError: lower_rank not defined

In [308]:
idx = sortperm(collect(values(ranking)))
[collect(keys(ranking))[idx], collect(values(ranking))[idx]]

2-element Array{Array{Any,1},1}:
 ["MRGA", "MRG", "MGA", "MG", "MRA", "RGA", "MR", "GA", "MA", "RG", "M", "G", "RA", "A", "R"]
 [1.0, 2.4375, 2.5625, 4.0, 6.0625, 6.875, 7.3125, 8.3125, 8.5, 9.6875, 10.5, 10.875, 12.875, 14.0, 15.0]

In [207]:
results[exp_name]["MR"]["model"].physics.B

0.565

In [181]:
ECS(results[exp_name][["adapt"]]["model"])

LoadError: KeyError: key ["adapt"] not found

In [182]:
_vals

1-element Array{Float64,1}:
 69.71395633009375

In [25]:
min_value = Inf
for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded") & (value["NPV"] < min_value)
        min_value = value["NPV"]
    end
end

for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded")
        abs_value = Int64(round((value["NPV"])))
        rel_value = Int64(round((value["NPV"]/min_value - 1.)*100))
        print("$index, $abs_value, $rel_value%\n")
    else
        print("$index has no solution.\n")
    end
end


LoadError: KeyError: key "status" not found